# Libraries Import

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

import numpy as np # linear algebra
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns 

from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import VectorAssembler

# Read Files

In [2]:
m_food_retail_price_path = r'D:\ME\Project\RetailCommoditiesPrice\Monthly_Food_Retail_Prices.csv'
m_nonfood_retail_price_path = r'D:\ME\Project\RetailCommoditiesPrice\Monthly_Non_Food_Retail_Prices.csv'
w_food_retail_price_path = r'D:\ME\Project\RetailCommoditiesPrice\Weekly_Food_Retail_Prices.csv'
w_nonfood_retail_price_path = r'D:\ME\Project\RetailCommoditiesPrice\Weekly_Non_Food_Retail_Prices.csv'

In [3]:
conf = pyspark.SparkConf().setMaster("local[*]") \
        .set('spark.executor.heartbeatInterval', 10000) \
        .set('spark.network.timeout', 10000) \
        .set("spark.core.connection.ack.wait.timeout", "3600") \
        .set("spark.executor.memory", '15G') \
        .set("spark.driver.memory", '15G')

spark = SparkSession \
        .builder \
        .appName("Retail-Commodities-Price-PySpark") \
        .config(conf=conf) \
        .getOrCreate()

In [4]:
# Load datasets
m_food_retail_price_raw = spark.read.option("header",True).csv(m_food_retail_price_path)
m_nonfood_retail_price_raw = spark.read.option("header",True).csv(m_nonfood_retail_price_path)
w_food_retail_price_raw = spark.read.option("header",True).csv(w_food_retail_price_path)
w_nonfood_retail_price_raw = spark.read.option("header",True).csv(w_nonfood_retail_price_path)

# Defining Functions

## Metadata

In [5]:
def get_metadata(dataframe):
    cols = dataframe.columns
    print(" STATISTICAL SUMMARY ".center(125, '#'))
    print()
    dataframe.summary().show()
    print(" SIZE OF DATAFRAME ".center(125, '#'))
    print("Rows : ", dataframe.count())
    print("Columns : ", len(dataframe.columns))
    print(" SCHEMA OF THE DATASET ".center(125, '#'))
    dataframe.printSchema()
    print(" NULL VALUE COUNTS ".center(125, '#'))
    print()
    dataframe.select([count(when(col(c).isNull(), c)).alias(c) for c in cols]).show()
    dup_count = dataframe.count()-dataframe.distinct().count()
    print(f" Number of duplicate Rows : {dup_count} ".center(125, '='))
    print()
    print(' SAMPLE VALUES IN EVERY COLUMN '.center(125, '#'))
    for c in cols:
        print(c)
        col_list = [data[0] for data in dataframe.select(f'{c}').distinct().orderBy(col(f'{c}')).collect()]
        print('Number of Distinct Values : ', len(col_list))
        print(col_list[:100])
    print(" TYPES OF COLUMNS ".center(125, '#'))
    print()
    print("Categorical Columns : ", [item[0] for item in dataframe.dtypes if item[1].startswith('string')])
    print("Numerical Columns: ", [item[0] for item in dataframe.dtypes if (item[1].startswith('integer') | item[1].startswith('double'))])
    print()
    # print(" NEGATIVE-VALUED COLUMNS ".center(125, '#'))
    # print()
    # sql_expressions = [f'count(CASE WHEN ({col} < 0) THEN 1 END) AS {col}_count' for col in cols]
    # dataframe.selectExpr(sql_expressions).show()
    print(''.center(125, '#'))

# EDA

## Monthly Food Price

In [6]:
get_metadata(m_food_retail_price_raw)

#################################################### STATISTICAL SUMMARY ####################################################

+-------+--------------+--------------+---------+--------------+----------+--------+--------+-----------------+
|summary|         State|        Centre|Commodity|       Variety|      Unit|Category|    Date|     Retail Price|
+-------+--------------+--------------+---------+--------------+----------+--------+--------+-----------------+
|  count|       1147446|       1147446|  1137726|        941868|   1147446| 1147446| 1147446|           578525|
|   mean|          null|          null|     null|          null|      null|    null|    null|79.69144840758825|
| stddev|          null|          null|     null|          null|      null|    null|    null| 148.780858660517|
|    min|Andhra Pradesh|      Agartala|    Apple|Amul/Any Other|100gm. Pkt|    Food|APR-2001|             0.50|
|    25%|          null|          null|     null|          null|      null|    null|    n

# Preprocessing